## This nb is used to download data from influxdb into csv files

### Setup

In [1]:
from influxdb_client import InfluxDBClient as InfluxDBClientv2
from influxdb_client.client.write_api import SYNCHRONOUS
import time, csv

# configuration of the database
host = 'stivm0016.xaas.epfl.ch'
port = 48086
token = 'dGGmbSVDyWnv03ZS1DDyZvea9o5UkrFRbhMYXANJkXG0RjjIWU5ojSVVyGzrlrwBU93mJ-vKmJgnysUsFWtlKg=='
org = 'Bassenges'

# open a connection to the DB
url = f"http://{host}:{port}"
client = InfluxDBClientv2(url=url, token=token)
query_api = client.query_api()

### Configuration

In [ ]:
first_dt = "2024-11-01T00:01:00Z"
last_dt = "2024-11-15T16:00:00Z"
res = 60 # resolution in seconds
bucket = "a_sensing"
measurements = ["co2", "htr","tmp"]

### Querying

In [3]:
# get the data from influx following the configuration above. Measurements is a list of the measurements to filter on.
query = f'from(bucket:"{bucket}") |> range(start: {first_dt}, stop: {last_dt}) |> filter(fn: (r) => r["_measurement"] == "{measurements[0]}"'
for m in measurements[1:]:
    query += f' or r["_measurement"] == "{m}"'
query += ')'
# Implement the resolution to compute mean values
query += f' |> aggregateWindow(every: {res}s, fn: mean, createEmpty: false)'
print(query)
# Query this data to retrieve a single table (put None for empty tags)
tables = query_api.query(org=org, query=query)
#save the data as a commented csv file in the data/ folder. Put the current datetime _influxdb_data as the filename
current_time = time.strftime("%Y-%m-%d_%H-%M")
filename = f"data/{current_time}_influxdb_data.csv"

# Extract and save data to a CSV file
with open(filename, 'w', newline='') as csvfile:
    # Define the CSV writer
    csv_writer = csv.writer(csvfile)

    # Initialize a variable to keep track of the current headers
    current_headers = None

    # Process tables and records
    for table in tables:
        for record in table.records:
            # Convert record to a dictionary for structured data
            record_dict = record.values

            # Get the headers of the current record
            headers = record_dict.keys()

            # Write headers if they are different from the current headers
            if headers != current_headers:
                csv_writer.writerow(['#'])
                csv_writer.writerow(headers)
                current_headers = headers

            # Write the record values
            csv_writer.writerow(record_dict.values())

print(f"Data saved to {filename}")

from(bucket:"a_sensing") |> range(start: 2024-11-01T00:01:00Z, stop: 2024-11-15T16:00:00Z) |> filter(fn: (r) => r["_measurement"] == "co2" or r["_measurement"] == "htr" or r["_measurement"] == "tmp") |> aggregateWindow(every: 60s, fn: mean, createEmpty: false)
Data saved to data/2025-03-14_18-00_influxdb_data.csv
